In [ ]:
import sys
import os

# Pobieramy ścieżkę do katalogu głównego projektu (dwa poziomy wyżej: ../..)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))

# Dodajemy ją do ścieżek Pythona, jeśli jej tam nie ma
if project_root not in sys.path:
    sys.path.append(project_root)

In [ ]:
import json

from app.database.database import Database
from app.database.sentence_transformer_embedder import SentenceTransformerEmbedder
from app.handler.defaulthandler import PromptHandler
from app.interface.abstractmodel import AbstractModel
from app.prompts_classification.fact_or_generative_classifier import FactOrGenerativeClassifier

with open('programming_questions.json', 'r') as f:
	data = json.load(f)

data

In [ ]:
class MockLLM(AbstractModel):

	def generate_answer(self, prompt: str) -> str:
		return "Answer"


db = Database(SentenceTransformerEmbedder())

handler = PromptHandler(db, MockLLM(), FactOrGenerativeClassifier())

for question in data:
	print(f"Question: {question['original_question']}")
	db.insert(question["original_question"], question["cached_answer"])

In [ ]:
expected_hits = 0
got_hits = 0
expected_misses = 0
got_misses = 0

for question in data:
    for positive_similar in question["paraphrases"]:
        print(f"Checking paraphrase: {positive_similar}")

        _, is_cached, _ = handler.generate_answer(positive_similar, False)

        if is_cached:
            got_hits += 1
        expected_hits += 1

    for negative in question["hard_negatives"]:
        print(f"Checking negative: {negative}")

        _, is_cached, _ = handler.generate_answer(negative, False)

        if not is_cached:
            got_misses += 1
        expected_misses += 1

    print("-" * 40)
    print(f"Processed question group: {question['original_question']}")
    print(f"  > Hits (Cache Hits):   {got_hits}/{expected_hits}")
    print(f"  > Misses (Cache Miss): {got_misses}/{expected_misses}")
    print("-" * 40 + "\n")

print("=== FINAL SUMMARY ===")
print(f"Total Accuracy (Hits):   {got_hits}/{expected_hits} ({(got_hits/expected_hits)*100:.1f}%)")
print(f"Total Accuracy (Misses): {got_misses}/{expected_misses} ({(got_misses/expected_misses)*100:.1f}%)")